In [23]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from openai import OpenAI
from langchain_openai import ChatOpenAI
import os
from unstructured.partition.pdf import partition_pdf
from PyPDF2 import PdfReader, PdfWriter
from unstructured.documents.elements import Element,Text,Image,FigureCaption
import pytesseract
import base64
from IPython.display import display, Image as IPImage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever






In [ ]:
import os
data_path = "C:/Users/002PMB744/Desktop/RAG-Langchain/RAG-Langchain/Data"
# data_path_1= "RAG-Langchain/Data"
# LANGCHAIN_TRACING_V2 = os.environ['LANGCHAIN_TRACING_V2']
# LANGCHAIN_ENDPOINT = os.environ["LANGCHAIN_ENDPOINT"]
# LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
# OPENAI_API_KEY= os.environ['OPENAI_API_KEY']


In [25]:
llm = ChatOpenAI(model="gpt-4o-mini",
                temperature=0.7, # Controls randomness: higher values (e.g., 1.0) for more creative, lower (e.g., 0.2) for more deterministic
                top_p=1.0, # Nucleus sampling: controls diversity by sampling from the most probable tokens
                frequency_penalty=0.0, # Penalizes new tokens based on their existing frequency in the text so far
                presence_penalty=0.0, # Penalizes new tokens based on whether they appear
                logit_bias={}, # Modifies the likelihood of specific tokens appearing in the output\
                response_format={"type": "text"}, # Specifies the format of the output (e.g., "text", "json_object")
                seed=None, # For reproducible outputs
                tools=[], # List of tools the model can call
                tool_choice="auto", # Controls how the model uses tools
                logprobs=False, # Whether to return log probabilities of output tokens
                top_logprobs=None # Number of top log probabilities to return
                 )

c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! tools is not default parameter.
                tools was transferred to model_kwargs.
                Please confirm that tools is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! tool_choice is not default parameter.
                tool_choice was transferred to model_kwargs.
                Please confirm that tool_choice is what you intended.
  if await 

In [26]:
print(llm.invoke("why do narendra modi lie so much?"))

content="Criticism of political leaders, including Narendra Modi, often stems from their statements or actions that some people perceive as misleading or false. Reasons behind such perceptions can include:\n\n1. **Political Strategy**: Politicians may exaggerate achievements or downplay failures to maintain support and bolster their public image.\n\n2. **Misinformation**: In an era of rapid information sharing, statements can be taken out of context or misrepresented.\n\n3. **Partisan Perspectives**: Supporters and critics often interpret statements differently based on their political allegiance.\n\n4. **Media Influence**: The way media outlets report on a leader's statements can shape public perception, leading some to believe that a leader is lying.\n\n5. **Complex Issues**: Political issues are often complex, and simplifications can lead to misunderstandings about what a leader actually said or meant.\n\nIt's important for citizens to critically evaluate information from various so

## Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process. These elements can be: Text, Images, Tables, etc.

In [27]:


# Define the PDF file and desired page numbers

base_dir = data_path
pdf_file_name = "Hands On Machine Learning with Scikit Learn and TensorFlow.pdf"
# pdf_file_name = "NIPS-2017-attention-is-all-you-need-Paper.pdf"
input_pdf_path= f"{base_dir}/{pdf_file_name}"
# input_pdf_path = "path/to/your/document.pdf"
output_dir = "./output_pages"
page_start = 275 +22
page_end = 311 +22  # For pages 3, 4, and 5

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Split the PDF into a smaller file with only the desired pages
output_pdf_path = os.path.join(output_dir, f"pages_{page_start}_to_{page_end}.pdf")
output_path = "./content/"
pdf_writer = PdfWriter()
pdf_reader = PdfReader(input_pdf_path)

for page_num in range(page_start - 1, page_end):
    pdf_writer.add_page(pdf_reader.pages[page_num])

with open(output_pdf_path, "wb") as out_file:
    pdf_writer.write(out_file)

# Partition the newly created smaller PDF
raw_chunks = partition_pdf(
    filename=output_pdf_path,
    # filename=pdf_file_path,
    strategy="hi_res",
    infer_table_structure=True,
    extract_image_block_types=["Image","Figure","Table"],
    image_output_dir_path=output_path,
    extract_image_block_to_payload=True,
    chunking_strategy=None,
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,
    poppler_path = r'C:\AppData\Roaming\Release-24.07.0-0\poppler-24.07.0\Library\bin'
)



KeyboardInterrupt: 

In [ ]:
# We get 2 types of elements from the partition_pdf function
set([str(type(el)) for el in raw_chunks])

{"<class 'unstructured.documents.elements.FigureCaption'>",
 "<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Formula'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [ ]:
# Each CompositeElement containes a bunch of related elements.
# This makes it easy to use these elements together in a RAG pipeline.
chunk_3 = raw_chunks[3].to_dict()
chunk_3['metadata']

{'detection_class_prob': 0.9302425384521484,
 'coordinates': {'points': ((np.float64(216.6841583251953),
    np.float64(911.1228637695312)),
   (np.float64(216.6841583251953), np.float64(1016.1986111111111)),
   (np.float64(1210.255126953125), np.float64(1016.1986111111111)),
   (np.float64(1210.255126953125), np.float64(911.1228637695312))),
  'system': 'PixelSpace',
  'layout_width': 1400,
  'layout_height': 1838},
 'last_modified': '2025-11-08T18:15:39',
 'filetype': 'application/pdf',
 'languages': ['eng'],
 'page_number': 1,
 'file_directory': './output_pages',
 'filename': 'pages_297_to_333.pdf',
 'parent_id': '2f6c211ad95eaa30ac640d29deb4088f'}

In [ ]:
# separate tables from texts
tables = []
texts = []
images = []

for chunk in raw_chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "Text" in str(type((chunk))):
        texts.append(chunk)

for idx, chunk in enumerate(raw_chunks):
    if isinstance(chunk, Image):
        # check idx+ 1 is a figure caption
        if idx + 1 < len(raw_chunks) and isinstance(raw_chunks[idx + 1], FigureCaption):
            caption = raw_chunks[idx + 1].text
        else:
            caption = None
        images.append(
            # "index": idx,
            # "image_text": chunk.text,
            # "caption": caption if caption else "No Caption",
            chunk.metadata.image_base64,
        )




In [ ]:


# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
# model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | llm | StrOutputParser()

In [ ]:
# Summarize text
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})
text_summaries

['Unable to access external websites. Please provide the table or text for summarization.',
 'Training deep neural networks (DNNs) with multiple layers, such as 10 layers containing hundreds of neurons, is essential for complex tasks like detecting various objects in high-resolution images, but it presents significant challenges.',
 'The chapter addresses the vanishing gradients problem and its solutions, discusses various optimizers that enhance training speed for large models compared to standard Gradient Descent, and reviews several popular regularization techniques for large neural networks.',
 'The text introduces tools that enable the training of very deep neural networks in the field of Deep Learning.',
 'The backpropagation algorithm computes the error gradient by propagating from the output layer to the input layer, calculating the gradient of the cost function for each layer.',
 '275 is a numerical value that could represent various things depending on the context, such as a 

In [ ]:
# Summarize tables
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 3})
table_summaries

['The table lists methods along with a numerical value of 2 for each: "rgistic," "yperbolic tangent," and "LU (and its variants)." Additionally, there are formulas related to inputs and outputs associated with these methods.',
 'The table outlines the following settings for a neural network: He initialization with ELU activation function, batch normalization, dropout for regularization, Adam optimizer, and no learning rate schedule.']

In [ ]:
# Summarize images

prompt_template = """Describe the image in detail. For context,
                  the image is part of a research paper explaining the transformers
                  architecture. Be specific about graphs, such as bar plots."""
messages = [
    (
        "user",
        [
            {"type": "text", "text": prompt_template},
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image}"},
            },
        ],
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | llm | StrOutputParser()


image_summaries = chain.batch(images)
print(image_summaries[2])

The image depicts a graph representing the Leaky ReLU (Rectified Linear Unit) activation function, commonly used in neural networks. 

### Key Features of the Graph:

1. **Axes**: 
   - The horizontal axis (x-axis) ranges from approximately -4 to 4, indicating the input values to the activation function.
   - The vertical axis (y-axis) ranges from 0 to 4, representing the output values of the Leaky ReLU function.

2. **Function Behavior**:
   - For negative input values (from -4 to 0), the graph shows a line with a slight upward slope, indicating that the function allows a small, non-zero output (the "leak") for negative inputs. This contrasts with the standard ReLU, which outputs zero for negative values.
   - At the point of zero input (0 on the x-axis), there is a transition in the slope.
   - For positive input values (from 0 to 4), the graph exhibits a straight line with a slope of 1, indicating that the output equals the input.

3. **Labeling**:
   - The graph is titled "Leaky Re

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

C:\Users\002PMB744\AppData\Local\Temp\ipykernel_23176\3922053165.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=OpenAIEmbeddings())
C:\Users\002PMB744\AppData\Local\Temp\ipykernel_23176\3922053165.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="multi_modal_rag", embeddi

In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
# print(summary_texts[2].page_content,summary_texts[2].metadata)
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=summary, metadata={id_key: table_ids[i]}) for i, summary in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in images]
summary_img = [
    Document(page_content=summary, metadata={id_key: img_ids[i]}) for i, summary in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, images)))

In [ ]:
# Retrieve
docs = retriever.invoke(
    "who is Xavier and He Initialization?"
)
for doc in docs:
    print(str(doc) + "\n\n" + "-" * 80)

Using the Xavier initialization strategy can speed up training considerably, and it is one of the tricks that led to the current success of Deep Learning. Some recent papers4 have provided similar strategies for different activation functions, as shown in Table 11-1. The initialization strategy for the ReLU activation function (and its var‐ iants, including the ELU activation described shortly) is sometimes called He initiali‐ zation (after the last name of its author).

--------------------------------------------------------------------------------
Equation 11-1. Xavier initialization (when using the logistic activation function)

--------------------------------------------------------------------------------
By default, the fully_connected() function (introduced in Chapter 10) uses Xavier initialization (with a uniform distribution). You can change this to He initialization by using the variance_scaling_initializer() function like this:

--------------------------------------------

## RAG pipeline

In [28]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode


def parse_docs(docs):
    """Split base64-encoded images and texts"""
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {"images": b64, "texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text

    # construct prompt with context (including images)
    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and the below image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | ChatOpenAI(model="gpt-4o-mini")
        | StrOutputParser()
    )
)

In [31]:
response = chain.invoke(
    "What is Xavier initialization?"
)

In [32]:
response

'Xavier initialization, also known as Glorot initialization, is a strategy for initializing the weights of layers in a neural network. It aims to improve the flow of signals during both forward predictions and backward gradient propagation. According to the authors Glorot and Bengio, this initialization helps maintain equal variance of the outputs and gradients before and after passing through a layer. It requires that the weights be initialized randomly based on the number of input connections (fan-in) and output connections (fan-out) for the layer. This approach alleviates issues related to signal dying out or exploding, facilitating more effective training of deep learning models.'